**라이브러리 임포트**

In [ ]:
import os
import json
import random
import shutil
import cv2
import math
import copy

from google.colab import drive

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision import transforms, models, datasets
from torch.utils.data import DataLoader, Subset

**데이터셋 unzip**

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/CAB/CAB_dataset/JPEGImages
!unzip -qq "/content/drive/MyDrive/CAB/CAB_dataset/JPEGImages/images_bbox.zip"

/content/drive/MyDrive/CAB/CAB_dataset/JPEGImages


In [ ]:
%cd /content/drive/MyDrive/CAB/CAB_dataset/Annotations
!unzip -qq "/content/drive/MyDrive/CAB/CAB_dataset/Annotations/label_bbox.zip"

/content/drive/MyDrive/CAB/CAB_dataset/Annotations


**시드 고정**

In [ ]:
torch.manual_seed(42)

**데이터 라벨링**

파일 경로 변환
- /content/drive/MyDrive/CAB/CAB_dataset/JPEGImages 하위 디렉토리에 있는 이미지를 /content/drive/MyDrive/CAB/CAB_dataset/JPEGImages로 옮긴다.
- /content/drive/MyDrive/CAB/CAB_dataset/Annotations 하위 디렉토리에 있는 이미지를 /content/drive/MyDrive/CAB/CAB_dataset/Annotations로 옮긴다.

In [ ]:
def move_file(base_dir):
  for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.lower().endswith(('.jpg', 'json')):
            source_path = os.path.join(root, file)
            destination_path = os.path.join(base_dir, file)

            shutil.move(source_path, destination_path)

jpeg_dir = "/content/drive/MyDrive/CAB/CAB_dataset/JPEGImages"
annot_dir = "/content/drive/MyDrive/CAB/CAB_dataset/Annotations"

move_file(jpeg_dir)
move_file(annot_dir)

입과 눈 사진을 저장할 폴더를 만들어 준다.

In [ ]:
data_root = "/content/drive/MyDrive/CAB/CAB_dataset/"
eyes_mouth = os.path.join(data_root, "EyesMouth")
os.makedirs(eyes_mouth, exist_ok=False)

In [ ]:
class_mapping = {
      "Face": 0,
      "Leye": 1,
      "Reye": 2,
      "Mouth": 3,
      "Cigar": 4,
      "Phone": 5
}

이미지에서 양쪽 눈과 입 사진을 잘라서 저장한다.
이때 파일명은 기존 파일명_클래스 이름_Open 혹은 기존 파일명_클래스 이름_Close로 한다.

In [ ]:
for image in os.listdir(jpeg_dir):
  if image.endswith('jpg'):
    name = os.path.splitext(image)[0]

    img_path = os.path.join(jpeg_dir, image)
    annot_path = os.path.join(annot_dir, name + '.json')

    with open(annot_path, 'r', encoding='utf-8') as f:
      data = json.load(f)

    for obj, bbox in data["ObjectInfo"]["BoundingBox"].items():
      if bbox["isVisible"] and class_mapping[obj] in [1, 2, 3]:

        x1, y1, x2, y2 = bbox["Position"]

        Is_open = "Open" if bbox["Opened"] else "Close"

        img = cv2.imread(img_path)
        crop_img = img[y1:y2, x1:x2]

        img_dst = os.path.join(eyes_mouth, f"{name}_{obj}_{Is_open}.jpg")
        cv2.imwrite(img_dst, crop_img)

파일명을 참고하여 이미지를 open 혹은 close 폴더로 옮긴다.



```python
# /content/drive/MyDrive/CAB/CAB_dataset/fs/
# ├── open/
# │   └── 파일명_클래스 이름_Open.jpg
# ├── close/
#      └── 파일명_클래스 이름_Close.jpg
```

In [ ]:
data_root = "/content/drive/MyDrive/CAB/CAB_dataset/"

fs_root = os.path.join(data_root, "fs")
open_root = os.path.join(fs_root, "open")
close_root = os.path.join(fs_root, "close")

os.makedirs(open_root, exist_ok=False)
os.makedirs(close_root, exist_ok=False)

In [ ]:
for file in os.listdir(eyes_mouth):
  if file.endswith('.jpg') and 'Open' in file:
    img_src = os.path.join(eyes_mouth, file)
    img_dst = os.path.join(open_root, file)

  elif file.endswith('.jpg') and 'Close' in file:
    img_src = os.path.join(eyes_mouth, file)
    img_dst = os.path.join(close_root, file)

  shutil.copy2(img_src, img_dst)


NameError: name 'eyes_mouth' is not defined

**데이터 undersampling**

다음으로, 모든 Image의 절대 경로가 적힌 리스트를 만든다.

In [ ]:
data_root = "/content/drive/MyDrive/CAB/CAB_dataset/"

fs_root = os.path.join(data_root, "fs")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(degrees=(30)),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size = 5, sigma = (0.5, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageFolder(root=fs_root, transform=transform)

class_to_idx = dataset.class_to_idx
print(f'클래스별 인덱스: {class_to_idx}')

class_counts = [0] * len(class_to_idx)
for _, target in dataset.samples:
    class_counts[target] += 1

print(f"클래스별 샘플 수: {class_counts}")
print(f"데이터셋 크기: {len(dataset)}")


클래스별 인덱스: {'close': 0, 'open': 1}
클래스별 샘플 수: [8661, 15364]
데이터셋 크기: 24025


추가 코드

close eye와 close mouth의 샘플 수 비율은 비슷하지만 open eye 샘플 수가 open mouth 샘플 수의 약 6배이다.

In [ ]:
close_samples = []
open_samples = []

for sample, target in dataset.samples:
  if target == 0:
    close_samples.append((sample, target))
  else:
    open_samples.append((sample, target))

print(f"close samples: {len(close_samples)}")
print(f"open samples: {len(open_samples)}")

close samples: 8661
open samples: 15364


데이터 불균형 문제와 제공된 GPU RAM, 학습 속도 등을 고려하여 클래스별 샘플 수를 줄여준다.

한 가지 추가적으로 고려해야 할 점은, 각 class 안의 eye sample과 mouth sample의 비율이다.

In [ ]:
e_close = []
e_open = []
m_close = []
m_open = []

In [ ]:
for sample, target in dataset.samples :
  if target == 0:
    if 'eye' in sample:
      e_close.append((sample, target))
    if 'Mouth' in sample:
      m_close.append((sample, target))
  else:
    if 'eye' in sample:
      e_open.append((sample, target))
    if 'Mouth' in sample:
      m_open.append((sample, target))

보다시피, open class 안에 eye sample이 mouth sample보다 약 6배는 더 많은 것을 알 수 있다.

In [ ]:
print(f"close eye samples: {len(e_close)}")
print(f"close mouth samples: {len(m_close)}")
print(f"open eye samples: {len(e_open)}")
print(f"open mouth samples: {len(m_open)}")

close eye samples: 4593
close mouth samples: 4068
open eye samples: 12898
open mouth samples: 2466


따라서 각 class의 비율뿐만 아니라 class 안의 mouth, eye sample이 비율도 고려하여 dataset을 나눈다.

모델의 최종 inference 결과를 봤을 때, 졸음에 대한 recall 값이 상당히 낮았다. 따라서 각 class에 입 사진보다 눈 사진을 더 많이 포함하였다.

In [ ]:
random.shuffle(e_close)
random.shuffle(e_open)
random.shuffle(m_close)
random.shuffle(m_open)

In [ ]:
close_samples = e_close[:1000] + e_close[3343:] + m_close[:1750]
open_samples = e_open[:1000] + e_open[11648:] + m_open[:1750]

In [ ]:
dataset.samples = close_samples + open_samples

**train/val/test loader 생성**

전체 이미지를 6:2:2 비율로 나누어 train set, validation set, test set을 생성한다.

ImageFolder는 instance를 직접 섞는 걸 허용하지 않는다. 따라서 무작위로 섞인 인덱스를 이용하여 훈련/검증/테스트 데이터셋을 만든다.


In [ ]:
dataset_indices = list(range(len(dataset)))

random.shuffle(dataset_indices)

train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = val_size

train_dataset = Subset(dataset, dataset_indices[:train_size])
val_dataset = Subset(dataset, dataset_indices[train_size:train_size+val_size])
test_dataset = Subset(dataset, dataset_indices[train_size+val_size:])

In [ ]:
print("length of train dataset: ", len(train_dataset))
print("length of val dataset: ", len(val_dataset))
print("length of test dataset: ", len(test_dataset))

length of train dataset:  4800
length of val dataset:  1600
length of test dataset:  1600


다음으로, 훈련/검증/테스트 데이터로더를 생성한다.

- num_workers: 2개의 프로세서가 병렬로 데이터를 불러와 이력 데이터가 더 빨리 준비될 수 있도록 한다.
- pin_memory=True: CPU에서 GPU로 데이터를 전송할 때 발생하는 복사 작업을 빠르게 할 수 있도록 도와준다.

In [ ]:
# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=2)

**분류 모델 생성**

분류 모델로는 이전 프로젝트에서 우수한 성능을 보였던 ResNet50 모델을 선택하였다.

다만 이번 프로젝트에서는 ResNet50 모델에 compound scaling을 적용해 보았다.

In [ ]:
# SE block + compound scaling을 적용한 ResNet50 훈련


# class SEBlock(nn.Module):
#     def __init__(self, in_channels, reduction=16):
#         super(SEBlock, self).__init__()
#         self.pool = nn.AdaptiveAvgPool2d(1)
#         self.conv1 = nn.Conv2d(in_channels, in_channels // reduction, kernel_size=1)
#         self.activ = nn.ReLU(inplace=True)
#         self.conv2= nn.Conv2d(in_channels // reduction, in_channels, kernel_size=1)
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         w = self.pool(x)
#         w = self.conv1(w)
#         w = self.activ(w)
#         w = self.conv2(w)
#         w = self.sigmoid(w)
#         x = x * w

#         return x

In [ ]:
# class CompoundScaledResNet50(nn.Module):
#   def __init__(self, width_mult=1.1, depth_mult=1.2, resolution_mult=1.15):
#     super(CompoundScaledResNet50, self).__init__()
#     self.base_model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

#     self.width_mult = width_mult
#     self.depth_mult = depth_mult
#     self.resolution_mult = resolution_mult

#     self.input_resolution = int(224 * resolution_mult) # increase the resolution of images

#     #self._modify_initial_layers()
#     self._apply_width_scaling_to_bottlenecks()
#     # self._modify_downsample_layers()
#     self._apply_depth_scaling()

#   def _apply_width_scaling_to_bottlenecks(self):
#     for name, module in self.base_model.named_modules():
#       if isinstance(module, models.resnet.Bottleneck):

#         orig_conv2 = module.conv2 # middle layer of bottleneck

#         scaled_channels = int(orig_conv2.out_channels * self.width_mult) # increase the channel of images

#         new_conv1 = nn.Conv2d(
#             in_channels = module.conv1.in_channels, # keep original input
#             out_channels = scaled_channels,
#             kernel_size = 1,
#             stride = module.conv1.stride,
#             bias = False
#         )

#         new_conv2 = nn.Conv2d(
#             in_channels = scaled_channels,
#             out_channels = scaled_channels,
#             kernel_size = 3,
#             stride = module.conv2.stride,
#             padding = 1,
#             bias = False
#         )

#         new_conv3 = nn.Conv2d(
#             in_channels = scaled_channels,
#             out_channels = module.conv3.out_channels,
#             kernel_size = 1,
#             bias = False
#         )

#         new_weight1 = torch.randn(scaled_channels, module.conv1.in_channels, 1, 1)
#         new_weight2 = torch.randn(scaled_channels, scaled_channels, 3, 3)
#         new_weight3 = torch.randn(module.conv3.out_channels, scaled_channels, 1, 1)

#         init.kaiming_uniform_(new_weight1, a=math.sqrt(5))
#         init.kaiming_uniform_(new_weight2, a=math.sqrt(5))
#         init.kaiming_uniform_(new_weight3, a=math.sqrt(5))

#         new_conv1.weight.data = new_weight1
#         new_conv2.weight.data = new_weight2
#         new_conv3.weight.data = new_weight3

#         new_bn1 = nn.BatchNorm2d(scaled_channels)
#         new_bn2 = nn.BatchNorm2d(scaled_channels)
#         new_bn3 = nn.BatchNorm2d(module.conv3.out_channels)

#         module.conv1 = new_conv1
#         module.bn1 = new_bn1

#         module.conv2 = new_conv2
#         module.bn2 = new_bn2

#         module.conv3 = new_conv3
#         module.bn3 = new_bn3

#         if (module.downsample):
#           downsample_module = module.downsample
#           del module.downsample

#           module.se_block = SEBlock(module.conv3.out_channels)
#           module.downsample = downsample_module

#         else:
#           module.se_block = SEBlock(module.conv3.out_channels)

#   def _modify_downsample_layers(self):
#     for name, module in self.base_model.named_modules():
#       if isinstance(module, models.resnet.Bottleneck) and module.downsample is not None:
#         conv1_in_channels = module.conv1.in_channels

#         if hasattr(module, 'downsample'):
#           module.downsample = nn.Sequential(
#               nn.Conv2d(
#                   in_channels = conv1_in_channels,
#                   out_channels = module.downsample[0].out_channels,
#                   kernel_size = 1,
#                   stride = module.downsample[0].stride,
#                   bias = False
#               ),
#               nn.BatchNorm2d(module.downsample[0].out_channels)
#           )

#   def _apply_depth_scaling(self):
#     for name, child in self.base_model.named_children():
#       if isinstance(child, nn.Sequential):
#         new_depth = math.ceil(len(child) * self.depth_mult) # increase the depth

#         if new_depth > len(child):
#           additional_layers = []

#           for _ in range(new_depth - len(child)):
#             new_layer = self._modify_for_dilated_conv(copy.deepcopy(child[-1]))
#             additional_layers.append(new_layer)

#           setattr(self.base_model, name, nn.Sequential(*list(child), *additional_layers)) # original seq + added layers


#   def _modify_for_dilated_conv(self, bottleneck, dilation=2):
#     b = bottleneck

#     if isinstance(b.conv2, nn.Conv2d):
#       b.conv2 = nn.Conv2d(
#           in_channels = b.conv2.in_channels,
#           out_channels = b.conv2.out_channels,
#           kernel_size = b.conv2.kernel_size,
#           stride = b.conv2.stride,
#           padding = (b.conv2.kernel_size[0] // 2) * dilation,
#           dilation = dilation
#       )
#     return b

#   def forward(self, x):
#     x = nn.functional.interpolate(x, size=(self.input_resolution, self.input_resolution), mode='bilinear')

#     return self.base_model(x)

In [ ]:
class CompoundScaledResNet50(nn.Module):
  def __init__(self, width_mult=1.1, depth_mult=1.2, resolution_mult=1.15):
    super(CompoundScaledResNet50, self).__init__()
    self.base_model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

    self.width_mult = width_mult
    self.depth_mult = depth_mult
    self.resolution_mult = resolution_mult

    self.input_resolution = int(224 * resolution_mult) # increase the resolution of images

    #self._modify_initial_layers()
    self._apply_width_scaling_to_bottlenecks()
    # self._modify_downsample_layers()
    self._apply_depth_scaling()

  def _apply_width_scaling_to_bottlenecks(self):
    for name, module in self.base_model.named_modules():
      if isinstance(module, models.resnet.Bottleneck):

        orig_conv2 = module.conv2 # middle layer of bottleneck

        scaled_channels = int(orig_conv2.out_channels * self.width_mult) # increase the channel of images

        new_conv1 = nn.Conv2d(
            in_channels = module.conv1.in_channels, # keep original input
            out_channels = scaled_channels,
            kernel_size = 1,
            stride = module.conv1.stride,
            bias = False
        )

        new_conv2 = nn.Conv2d(
            in_channels = scaled_channels,
            out_channels = scaled_channels,
            kernel_size = 3,
            stride = module.conv2.stride,
            padding = 1,
            bias = False
        )

        new_conv3 = nn.Conv2d(
            in_channels = scaled_channels,
            out_channels = module.conv3.out_channels,
            kernel_size = 1,
            bias = False
        )

        new_weight1 = torch.randn(scaled_channels, module.conv1.in_channels, 1, 1)
        new_weight2 = torch.randn(scaled_channels, scaled_channels, 3, 3)
        new_weight3 = torch.randn(module.conv3.out_channels, scaled_channels, 1, 1)

        init.kaiming_uniform_(new_weight1, a=math.sqrt(5))
        init.kaiming_uniform_(new_weight2, a=math.sqrt(5))
        init.kaiming_uniform_(new_weight3, a=math.sqrt(5))

        new_conv1.weight.data = new_weight1
        new_conv2.weight.data = new_weight2
        new_conv3.weight.data = new_weight3

        new_bn1 = nn.BatchNorm2d(scaled_channels)
        new_bn2 = nn.BatchNorm2d(scaled_channels)
        new_bn3 = nn.BatchNorm2d(module.conv3.out_channels)

        module.conv1 = new_conv1
        module.bn1 = new_bn1

        module.conv2 = new_conv2
        module.bn2 = new_bn2

        module.conv3 = new_conv3
        module.bn3 = new_bn3

  def _modify_downsample_layers(self):
    for name, module in self.base_model.named_modules():
      if isinstance(module, models.resnet.Bottleneck) and module.downsample is not None:
        conv1_in_channels = module.conv1.in_channels

        if hasattr(module, 'downsample'):
          module.downsample = nn.Sequential(
              nn.Conv2d(
                  in_channels = conv1_in_channels,
                  out_channels = module.downsample[0].out_channels,
                  kernel_size = 1,
                  stride = module.downsample[0].stride,
                  bias = False
              ),
              nn.BatchNorm2d(module.downsample[0].out_channels)
          )

  def _apply_depth_scaling(self):
    for name, child in self.base_model.named_children():
      if isinstance(child, nn.Sequential):
        new_depth = math.ceil(len(child) * self.depth_mult) # increase the depth

        if new_depth > len(child):
          additional_layers = []

          for _ in range(new_depth - len(child)):
            new_layer = self._modify_for_dilated_conv(copy.deepcopy(child[-1]))
            additional_layers.append(new_layer)

          setattr(self.base_model, name, nn.Sequential(*list(child), *additional_layers)) # original seq + added layers


  def _modify_for_dilated_conv(self, bottleneck, dilation=2):
    b = bottleneck

    if isinstance(b.conv2, nn.Conv2d):
      b.conv2 = nn.Conv2d(
          in_channels = b.conv2.in_channels,
          out_channels = b.conv2.out_channels,
          kernel_size = b.conv2.kernel_size,
          stride = b.conv2.stride,
          padding = (b.conv2.kernel_size[0] // 2) * dilation,
          dilation = dilation
      )
    return b

  def forward(self, x):
    x = nn.functional.interpolate(x, size=(self.input_resolution, self.input_resolution), mode='bilinear')

    return self.base_model(x)

In [ ]:
class MyCompoundScaledResNet50(nn.Module):
  def __init__(self, width_mult=1.1, depth_mult=1.2, resolution_mult=1.15):
    super(MyCompoundScaledResNet50, self).__init__()

    self.backbone = CompoundScaledResNet50(
        width_mult=width_mult,
        depth_mult=depth_mult,
        resolution_mult=resolution_mult
    )

    self.dropout = nn.Dropout(0.3)
    self.extra_layer = nn.Linear(1000, 2)

  def forward(self, x):
    x = self.backbone(x)
    x = self.dropout(x)
    x = self.extra_layer(x)
    return x

In [ ]:
c_resnet = MyCompoundScaledResNet50().cuda()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 96.0MB/s]


In [ ]:
# "modules"는 자신에게 속하는 모든 submodule들을 표시
# "children"은 한 단계 아래의 submodule까지만 표시
# https://data-scientist-han.tistory.com/110

for name, module in c_resnet.named_modules():
    print(name, module)
    break

 MyCompoundScaledResNet50(
  (backbone): CompoundScaledResNet50(
    (base_model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 70, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(70, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(70, 70, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(70, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(70, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

학습이 불안정할 때 더 빠르고 안정적인 수렴을 유도하기 위해 NAdam optimizer를 사용하였고, 학습 후반부에 과적합을 방지하면서 세밀하게 학습할 수 있도록 CosineAnnealingLR도 사용하였다.

또한 모델의 일반화 성능을 향상시키기 위해 l2 정규화를 적용하였다.

In [ ]:
optimizer = optim.NAdam(c_resnet.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40, eta_min=0)

criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**훈련/검증/테스트 함수 정의**

In [ ]:
def train(model):
  model.train()

  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)

    optimizer.zero_grad()

    output = model(data)

    loss = criterion(output, target)
    loss.backward()

    optimizer.step()
  scheduler.step()

In [ ]:
def val(model, epoch):
  model.eval()

  val_loss = 0
  correct = 0

  with torch.no_grad():
    for data, target in val_loader:
      data, target = data.to(device), target.to(device)

      output = model(data)

      val_loss += criterion(output, target).item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  val_loss /= len(val_loader.dataset)
  accuracy = 100. * correct / len(val_loader.dataset)

  print(f"Epoch: {epoch}, Average loss: {val_loss:.4f}, Accuracy: {correct}/{len(val_loader.dataset)} ({accuracy:.2f}%)")

  return val_loss

In [ ]:
def test(model):
  model.eval()

  test_loss = 0
  correct = 0

  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)

      output = model(data)

      test_loss += criterion(output, target).item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  accuracy = 100. * correct / len(test_loader.dataset)

  print(f"Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)")


**EarlyStopping 정의**

overfitting을 방지하기 위해 earlystopping을 사용한다.

In [ ]:
class EarlyStopping:
    def __init__(self, patience=13, verbose=False, counter=0, best_loss=float('inf')):
        self.patience = patience
        self.verbose = verbose
        self.counter = counter
        self.best_loss = best_loss
        self.early_stop = False

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.counter = 0
            self.best_loss = val_loss
            torch.save(model.state_dict(), '/content/drive/MyDrive/CAB/CAB_dataset/model/best_decay_model.pth')
        else:
            self.counter += 1
            if self.counter >= self.patience:
              self.early_stop = True

**모델 학습 및 평가**

In [ ]:
num_epochs = 200

In [ ]:
early_stopping = EarlyStopping(verbose=True)

for epoch in range(1, num_epochs + 1):
  train(c_resnet)
  val_loss = val(c_resnet, epoch)

  early_stopping(val_loss, c_resnet)

  if early_stopping.early_stop:
    print("************************************************************\nEarly stop!")
    c_resnet.load_state_dict(torch.load('/content/drive/MyDrive/CAB/CAB_dataset/model/best_decay_model.pth', map_location=device))
    test(c_resnet)
    break

Epoch: 1, Average loss: 0.0212, Accuracy: 1065/1600 (66.56%)
Epoch: 2, Average loss: 0.0341, Accuracy: 1010/1600 (63.12%)
Epoch: 3, Average loss: 0.0162, Accuracy: 1230/1600 (76.88%)
Epoch: 4, Average loss: 0.0167, Accuracy: 1201/1600 (75.06%)
Epoch: 5, Average loss: 0.0180, Accuracy: 1140/1600 (71.25%)
Epoch: 6, Average loss: 0.0127, Accuracy: 1319/1600 (82.44%)
Epoch: 7, Average loss: 0.0306, Accuracy: 1097/1600 (68.56%)
Epoch: 8, Average loss: 0.0151, Accuracy: 1252/1600 (78.25%)
Epoch: 9, Average loss: 0.0313, Accuracy: 1109/1600 (69.31%)
Epoch: 10, Average loss: 0.0275, Accuracy: 1125/1600 (70.31%)
Epoch: 11, Average loss: 0.0125, Accuracy: 1330/1600 (83.12%)
Epoch: 12, Average loss: 0.0214, Accuracy: 1157/1600 (72.31%)
Epoch: 13, Average loss: 0.0116, Accuracy: 1337/1600 (83.56%)
Epoch: 14, Average loss: 0.0161, Accuracy: 1291/1600 (80.69%)
Epoch: 15, Average loss: 0.0100, Accuracy: 1388/1600 (86.75%)
Epoch: 16, Average loss: 0.0076, Accuracy: 1453/1600 (90.81%)
Epoch: 17, Averag

<ipython-input-26-e0aed74a9c65>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  c_resnet.load_state_dict(torch.load('/content/drive/MyDrive/CAB/CAB_dataset/model/best_deca

Average loss: 0.0043, Accuracy: 1529/1600 (95.56%)
